<a href="https://colab.research.google.com/github/IsabelleEve/NLP-Classification-Project-/blob/main/BERT_IMDB_Sentiment_Anaysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import tensorflow as tf
import os
!pip install -q opendatasets
import opendatasets as od
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('notebook',font_scale=1.25)
from wordcloud import WordCloud,STOPWORDS
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML,display
import re
import timeit
import gc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras import mixed_precision

!pip install datasets --quiet
!pip install transformers

import transformers
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding
from transformers import TFAutoModelForSequenceClassification

     |████████████████████████████████| 441 kB 13.8 MB/s 
     |████████████████████████████████| 163 kB 61.7 MB/s 
     |████████████████████████████████| 212 kB 63.0 MB/s 
     |████████████████████████████████| 115 kB 73.4 MB/s 
     |████████████████████████████████| 127 kB 71.3 MB/s 
     |████████████████████████████████| 115 kB 73.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 13.2 MB/s 
     |████████████████████████████████| 7.6 MB 64.4 MB/s 


In [ ]:
# function ClickConnect() {
# console.log("Working");
# document
#   .querySelector('#top-toolbar > colab-connect-button')
#   .shadowRoot.querySelector('#connect')
#   .click()
# }
# setInterval(ClickConnect, 60000)

In [ ]:
data=load_dataset('csv',data_files='/content/drive/MyDrive/movie.csv')
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f33ffea046a1490c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 40000
    })
})

In [ ]:
# converting to lowercase
def lowercase_text(example):
    return{'text':example['text'].lower()}
data=data.map(lowercase_text)

  0%|          | 0/40000 [00:00<?, ?ex/s]

In [ ]:
checkpoint='bert-base-uncased'
# checkpoint='roberta-large-mnli'

tokenizer=AutoTokenizer.from_pretrained(checkpoint)

#defining the tokenization function
def tokenize_function(example):
    return tokenizer(example['text'],truncation=True)

tokenized_dataset=data.map(tokenize_function,batched=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/40 [00:00<?, ?ba/s]

In [ ]:
#policy to be used while batching
data_collator=DataCollatorWithPadding(tokenizer=tokenizer,return_tensors='tf')

In [ ]:
tokenized_splitted_dataset=tokenized_dataset['train'].train_test_split(test_size=0.2)
test_set_lenght=len(tokenized_splitted_dataset['test']['label'])
tokenized_splitted_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
})

In [ ]:
tf_train_dataset = tokenized_splitted_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
).prefetch(tf.data.AUTOTUNE)

tf_validation_dataset = tokenized_splitted_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
).prefetch(tf.data.AUTOTUNE)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
model_6 = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
epochs=1
mini_batches=int(0.2*len(tf_train_dataset))
num_steps=epochs*mini_batches
learning_rate_schedule=PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=num_steps,
    end_learning_rate=0.0,
)

In [ ]:
model_6.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate_schedule),
    metrics=['accuracy']
)

In [ ]:
model_6_history=model_6.fit(
    tf_train_dataset,
    steps_per_epoch=int(0.2*len(tf_train_dataset)),
    epochs=epochs,
    validation_data=tf_validation_dataset
)

104/800 [==>...........................] - ETA: 6:36:39 - loss: 0.6951 - accuracy: 0.5036

In [ ]:
start_time=timeit.default_timer()
model_6_pred_logits=model_6.predict(tf_validation_dataset)
model_6_pred=tf.round(tf.nn.sigmoid(model_6_pred_logits['logits']))
end_time=timeit.default_timer()
time_taken_per_instance=(end_time-start_time)/test_set_lenght
model_6_results=binary_clf_evaluation(tokenized_splitted_dataset['test']['label'],model_6_pred,time_taken_per_instance)
display_results(model_6_results)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
while True:pass

# New Section